In [0]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
from torchvision.datasets import VisionDataset

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image




def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

class Caltech(VisionDataset):
    def __init__(self, root, split='train', transform=None, target_transform=None):
        super(Caltech, self).__init__(root, transform=transform, target_transform=target_transform)

        #self.split = split # This defines the split you are going to use
                           # (split files are called 'train.txt' and 'test.txt')
        self.root = root
 
        '''
        - Here you should implement the logic for reading the splits files and accessing elements
        - If the RAM size allows it, it is faster to store all data in memory
        - PyTorch Dataset classes use indexes to read elements
        - You should provide a way for the __getitem__ method to access the image-label pair
          through the index
        - Labels should start from 0, so for Caltech you will have lables 0...100 (excluding the background class) 
        '''
        z=0
        c=0
        
        dir = os.path.expanduser(root+'101_ObjectCategories/')
        l=[]
        self.samples=[]
        labels=[]
        n=[]
        
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]        
        classes.sort()
        print(len(classes),classes)
        class_to_idx = {classes[i]: i for i in range(len(classes))}
        del class_to_idx['BACKGROUND_Google']
        print(len(class_to_idx),class_to_idx)
        b = 0
        with open(root + split+'.txt','r') as f:
            for line in f:
                row = line.split("\n")[0]
                #empty = field[1]
                label = line.split("/")[0]
                if(label != 'BACKGROUND_Google'):
                    self.path = os.path.join(dir,row)
                    s = class_to_idx[label]
                    self.n=n
                    self.n.append(s)
                    self.samples.append((pil_loader(self.path),s))
                    c = c+1
                z+=1
        
        
        

        dim = self.__len__()
        f.close()
        
        print("iterations without BACKGROUND",c)
        print("iterations with BACKGROUND",z)
       
        print("Number of images of %s = %d "%(split, dim))
        print("Number of images of %s without BACKGROUND_Google Class = %d" %(split,c))      
        print("len",len(class_to_idx))


    def __getitem__(self, index):
        '''
        __getitem__ should access an element through its index
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        '''   
        #image, label =  
                           # Provide a way to access image and label via index
                           # Image should be a PIL Image
                           # label can be int

        image,label = self.samples[index]        #print("index",index)
        #print("label",label)
        #image,label = self.path,index
        #image = pil_loader(self.path)
        #image = pil_loader(path)

        # Applies preprocessing when accessing the image
        if self.transform is not None:
            image = self.transform(image)
            #print(image)
            #label=self.transform(label)
        if label>4048:
            label-=4048
        return image, label-1

    def __len__(self):
        '''
        The __len__ method returns the length of the dataset
        It is mandatory, as this is used by several other components
        '''
        print(len(self.n))
         # Provide a way to get the length (number of elements) of the dataset
        return len(self.n)



In [0]:
#SET ARGUMENTS
DEVICE = 'cuda' # 'cuda' or 'cpu'


NUM_CLASSES = 101 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 64  # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-2            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS =  30     # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

In [0]:
#PREPARE DATA PREPROCESSING
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                       
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

In [5]:
#PREPARE DATASET
# Clone github repository with data
if not os.path.isdir('./Caltech101'):
    !git clone https://github.com/sofiaborgato/Homework2-Caltech101.git
    os.rename('Homework2-Caltech101','Caltech101')

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

    


DATA_DIR = 'Caltech101/'
from torchvision.datasets import VisionDataset
from Caltech101.caltech_dataset import Caltech


# Prepare Pytorch train/test Datasets
first_train_dataset = Caltech(DATA_DIR, split='train',  transform=train_transform)
print(first_train_dataset.__len__())

test_dataset = Caltech(DATA_DIR, split='test', transform=eval_transform)

train_indexes = [i for i in range(len(first_train_dataset)) if i % 2]
print(train_indexes)
val_indexes = [i for i in range(len(first_train_dataset)) if not i % 2]
print(val_indexes)
train_dataset = Subset(first_train_dataset, train_indexes)
val_dataset = Subset(first_train_dataset, val_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Valid Dataset: {}'.format(len(val_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

102 ['BACKGROUND_Google', 'Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_horse', 'snoopy'

In [0]:
#PREPARE DATALOADERS
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
val_dataloader = DataLoader(val_dataset,batch_size=int(BATCH_SIZE/2), shuffle=False, num_workers=4)


test_dataloader = DataLoader(test_dataset,batch_size=int(BATCH_SIZE/2), shuffle=False, num_workers=4)

In [0]:
#PREPARE NETWORK
net = alexnet() # Loading AlexNet model

# AlexNet has 1000 output neurons, corresponding to the 1000 ImageNet's classes
# We need 101 outputs for Caltech-101
net.classifier[6] = nn.Linear(4096, NUM_CLASSES) # nn.Linear in pytorch is a fully connected layer
                                                 # The convolutional layer is nn.Conv2d

# We just changed the last layer of AlexNet with a new fully connected layer with 101 outputs
# It is strongly suggested to study torchvision.models.alexnet source code

In [0]:
#PREPARE TRAINING 

# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
#parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
#optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

In [9]:
#TRAIN WITH VALIDATION

net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
cudnn.benchmark = True # Calling this optimizes runtime
#Hyper-parameter to optimize
learning_rate_vector=[1e-2,1e-3]
epoch_vector=[50,30]
loss_val=[]
val_matrix=[[]]
loss_matrix=[[]]
acc_matrix=[[]]

current_step = 0
best_accuracy = 0
loss_vector = []
tot_accuracy = []
#iterate over configuration:
for lr,epoch_n in zip(learning_rate_vector,epoch_vector):
  parameters_to_optimize = net.parameters()
  optimizer = optim.SGD(parameters_to_optimize, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  print("try with=",lr)
  print("try with",epoch_n)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
  for epoch in range(epoch_n):
    print('Starting epoch {}/{}, LR = {}'.format(epoch+1, epoch_n, scheduler.get_lr()))

    # Iterate over the dataset
    for images, labels in train_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      net.train() # Sets module in training mode

      # PyTorch, by default, accumulates gradients after each backward pass
      # We need to manually set the gradients to zero before starting a new iteration
      optimizer.zero_grad() # Zero-ing the gradients

      # Forward pass to the network
      outputs = net(images)

      # Compute loss based on output and ground truth
      loss = criterion(outputs, labels)

      # Log loss
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1

    loss_vector.append(loss.item())

    # Step the scheduler
    scheduler.step()

    #VALIDATION
    net.train(False) # Set Network to evaluation mode
    loss_val.append(criterion(outputs,labels))

    running_corrects = 0
    with torch.no_grad():
        for images, labels in tqdm(val_dataloader):
          images = images.to(DEVICE)
          labels = labels.to(DEVICE)

          # Forward Pass
          outputs = net(images)

          # Get predictions
          _, preds = torch.max(outputs.data, 1)

          # Update Corrects
          running_corrects += torch.sum(preds == labels.data).data.item()
    # Calculate Accuracy
    accuracy = running_corrects / float(len(val_dataset))
    tot_accuracy.append(accuracy)
    print('Validation Accuracy: {}'.format(accuracy))

    if accuracy>best_accuracy:
      best_net = copy.deepcopy(net)
      best_accuracy=accuracy
  val_matrix.append(loss_val)
  loss_matrix.append(loss_vector)
  acc_matrix.append(tot_accuracy)
  loss_val=[]
  loss_vector=[]
  tot_accuracy=[]
     

try with= 0.01
try with 50
Starting epoch 1/50, LR = [0.01]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss 4.612905025482178
Step 10, Loss 4.5925798416137695
Step 20, Loss 4.5901408195495605
Step 30, Loss 4.558862209320068
Step 40, Loss 4.5228657722473145


100%|██████████| 91/91 [00:10<00:00,  8.79it/s]

Validation Accuracy: 0.09197786998616875
Starting epoch 2/50, LR = [0.01]


...

100%|██████████| 91/91 [00:10<00:00,  8.84it/s]

Validation Accuracy: 0.5663900414937759
Starting epoch 30/30, LR = [0.0001]


Step 3560, Loss 0.022392332553863525
Step 3570, Loss 0.0012768805027008057
Step 3580, Loss 0.010462373495101929
Step 3590, Loss 0.040920406579971313


100%|██████████| 91/91 [00:10<00:00,  8.71it/s]

Validation Accuracy: 0.5656984785615491


In [0]:
#tit=tit = 'LossFunction - BATCH_SIZE= %d LR= %d  EPOCHS= %d  STEP_SIZE= %d GAMMA= %d' % (BATCH_SIZE, lr, epoch, STEP_SIZE,GAMMA)
  #x=np.linespace(1,epoch,num=epoch)
  #plt.plot(x,loss_vector,color='blue')
  #plt.plot(x,loss_val,color='orange')
  #plt.title(tit)
  #plt.legend(['Loss_train','Loss_val'],loc='upper right')
  #plt.show()

  #plt.plot(tot_accuracy,color='red')
  #plt.title(tit

In [11]:
#TEST

net = best_net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False)

running_corrects = 0
for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

# Forward Pass
outputs = net(images)

# Get predictions
_, preds = torch.max(outputs.data, 1)

# Update Corrects
running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

  0%|          | 0/91 [00:00<?, ?it/s]

2893
2893


100%|██████████| 91/91 [00:10<00:00,  8.77it/s]

2893
Test Accuracy: 0.0034566194262011752
